In [8]:
import pandas as pd
import numpy as np
import mne
import os
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import ttk

class EEGDataExtractorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("EEG Data Extractor")
        self.root.geometry("800x600")  # 放大窗口大小

        # 创建一个框架来放置并排按钮
        button_frame = tk.Frame(root)
        button_frame.pack(pady=10)

        # 添加按钮
        self.load_tsv_button = tk.Button(button_frame, text="加载TSV文件", command=self.load_tsv_file)
        self.load_tsv_button.pack(side=tk.LEFT, padx=5)

        self.load_eeg_button = tk.Button(button_frame, text="选择EEG文件", command=self.load_eeg_file, state=tk.DISABLED)
        self.load_eeg_button.pack(side=tk.LEFT, padx=5)

        # 添加下拉菜单选择EEG数据格式
        self.eeg_format_label = tk.Label(root, text="选择EEG数据格式:")
        self.eeg_format_label.pack(pady=5)

        self.eeg_format = tk.StringVar()
        self.eeg_format.set(None)  # 默认无选择

        self.eeg_format_combobox = ttk.Combobox(root, textvariable=self.eeg_format, values=["edf", "fif", "vhdr", "bdf"])
        self.eeg_format_combobox.bind("<<ComboboxSelected>>", self.on_eeg_format_selected)
        self.eeg_format_combobox.pack(pady=5)

        # 添加按钮
        self.save_dir_button = tk.Button(root, text="选择保存目录", command=self.select_save_dir)
        self.save_dir_button.pack(pady=10)

        self.process_button = tk.Button(root, text="处理数据", command=self.process_data)
        self.process_button.pack(pady=10)

        self.close_button = tk.Button(root, text="关闭", command=self.close_app)
        self.close_button.pack(pady=10)

        # 用于存储保存目录
        self.save_dirs = {}

        self.tsv_file_path = None
        self.eeg_file_path = None
        self.output_dir = None

    def load_tsv_file(self):
        self.tsv_file_path = filedialog.askopenfilename(filetypes=[("TSV files", "*.tsv")])
        if self.tsv_file_path:
            self.show_tsv_preview()
            self.create_trial_type_buttons()
            messagebox.showinfo("信息", f"TSV 文件已加载: {self.tsv_file_path}")

    def show_tsv_preview(self):
        try:
            df = pd.read_csv(self.tsv_file_path, sep='\t')
            preview_data = df.head(10)
        except Exception as e:
            messagebox.showerror("错误", f"读取TSV文件时出现错误: {str(e)}")
            return

        # 创建一个新的窗口显示数据
        preview_window = tk.Toplevel(self.root)
        preview_window.title("TSV文件预览")
        preview_window.geometry("600x300")

        text_area = tk.Text(preview_window, wrap=tk.NONE)
        text_area.pack(expand=True, fill=tk.BOTH)

        text_area.insert(tk.END, preview_data.to_string(index=False))

    def create_trial_type_buttons(self):
        # 删除现有的 trial_type 按钮
        for widget in self.root.winfo_children():
            if isinstance(widget, tk.Button) and widget.cget("text").startswith("选择保存目录"):
                widget.destroy()

        if self.tsv_file_path:
            df = pd.read_csv(self.tsv_file_path, sep='\t')
            trial_types = df['trial_type'].unique()

            self.save_dirs = {}  # 重置保存目录字典

            for trial_type in trial_types:
                button = tk.Button(self.root, text=f"选择 {trial_type} 保存目录", command=lambda t=trial_type: self.select_save_dir(t))
                button.pack(pady=5)

    def select_save_dir(self, trial_type=None):
        if trial_type:
            dir_path = filedialog.askdirectory()
            if dir_path:
                self.save_dirs[trial_type] = dir_path
                messagebox.showinfo("信息", f"{trial_type} 保存目录已选择: {dir_path}")

    def load_eeg_file(self):
        eeg_format = self.eeg_format.get()
        if not eeg_format:
            messagebox.showerror("错误", "请先选择EEG数据格式")
            return

        filetypes = {
            'edf': ("EDF files", "*.edf"),
            'fif': ("FIF files", "*.fif"),
            'vhdr': ("BrainVision files", "*.vhdr"),
            'bdf': ("BDF files", "*.bdf")
        }
        
        selected_filetypes = filetypes.get(eeg_format, ("All files", "*.*"))
        self.eeg_file_path = filedialog.askopenfilename(filetypes=[selected_filetypes])

        if self.eeg_file_path:
            messagebox.showinfo("信息", f"EEG 文件已加载: {self.eeg_file_path}")

    def on_eeg_format_selected(self, event):
        # 当选择格式时启用加载EEG文件按钮
        self.load_eeg_button.config(state=tk.NORMAL)

    def process_data(self):
        if not self.tsv_file_path or not self.eeg_file_path or not self.save_dirs:
            messagebox.showerror("错误", "请确保已加载TSV文件、EEG文件，并选择所有试验类型的保存目录")
            return

        eeg_format = self.eeg_format.get()
        
        try:
            # 读取TSV文件
            events_df = pd.read_csv(self.tsv_file_path, sep='\t')
            onsets = events_df['onset'].values
            durations = events_df['duration'].values
            trial_types = events_df['trial_type'].values

            # 读取EEG数据
            if eeg_format == 'edf':
                eeg_data = mne.io.read_raw_edf(self.eeg_file_path, preload=True)
            elif eeg_format == 'fif':
                eeg_data = mne.io.read_raw_fif(self.eeg_file_path, preload=True)
            elif eeg_format == 'vhdr':
                eeg_data = mne.io.read_raw_brainvision(self.eeg_file_path, preload=True)
            elif eeg_format == 'bdf':
                eeg_data = mne.io.read_raw_bdf(self.eeg_file_path, preload=True)
            else:
                raise ValueError("不支持的EEG文件格式")

            sfreq = eeg_data.info['sfreq']

            # 提取并保存试验数据
            for i, (onset, duration, trial_type) in enumerate(zip(onsets, durations, trial_types)):
                start_idx = int(onset * sfreq)
                end_idx = int((onset + duration) * sfreq)
                trial_data = eeg_data.get_data(start=start_idx, stop=end_idx)

                # 使用 trial_type 来选择保存目录
                if trial_type in self.save_dirs:
                    file_name = f'trial_{i}_{trial_type}.npy'
                    file_path = os.path.join(self.save_dirs[trial_type], file_name)
                    np.save(file_path, trial_data)
                    print(f'Saved trial {i} ({trial_type}) to {file_path}')
                else:
                    messagebox.showwarning("警告", f"未为 trial_type '{trial_type}' 选择保存目录")

            messagebox.showinfo("完成", "所有试验数据已成功保存。")
        except Exception as e:
            messagebox.showerror("错误", f"处理数据时出现错误: {str(e)}")

    def close_app(self):
        self.root.destroy()

# 创建GUI应用程序
root = tk.Tk()
app = EEGDataExtractorApp(root)
root.mainloop()


Extracting EDF parameters from C:\Users\pc\Desktop\李佳\sub-001_ses-03_task_motorimagery_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 98999  =      0.000 ...   395.996 secs...
Saved trial 0 (right) to C:/Users/pc/Desktop/李佳/right\trial_0_right.npy
Saved trial 1 (left) to C:/Users/pc/Desktop/李佳/left\trial_1_left.npy
Saved trial 2 (right) to C:/Users/pc/Desktop/李佳/right\trial_2_right.npy
Saved trial 3 (right) to C:/Users/pc/Desktop/李佳/right\trial_3_right.npy
Saved trial 4 (right) to C:/Users/pc/Desktop/李佳/right\trial_4_right.npy
Saved trial 5 (right) to C:/Users/pc/Desktop/李佳/right\trial_5_right.npy
Saved trial 6 (left) to C:/Users/pc/Desktop/李佳/left\trial_6_left.npy
Saved trial 7 (left) to C:/Users/pc/Desktop/李佳/left\trial_7_left.npy
Saved trial 8 (left) to C:/Users/pc/Desktop/李佳/left\trial_8_left.npy
Saved trial 9 (right) to C:/Users/pc/Desktop/李佳/right\trial_9_right.npy
Saved trial 10 (left) to C:/Users/pc/Desktop/李佳/left\tr